In [ ]:
import networkx as nx
import numpy as np
import scipy.sparse as scisparse
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import sys
from pathlib import Path
pp = str(Path('.').absolute().parent)
if pp not in sys.path:
    sys.path.append(pp)

In [ ]:
from qpr.routing import NetList, Route
from qpr.graph_utils import get_paths_for_nodes, get_paths_for_nodes_bfs

In [ ]:
A = np.array([[0, 1, 1, 1], [1, 0, 1, 0], [0, 0, 0, 1], [1, 0, 0, 0]])
g4 = nx.DiGraph(A)

# insert a node in the middle of 2-3 edge but don't change its direction
A = np.array([[0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [0, 0, 0, 1, 0]])
g4_same = nx.DiGraph(A)

# insert a node in the middle of 2-3 and reverse it, i.e. isolate 2 from the rest of the (directed) graph
A = np.array([[0, 1, 1, 1, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 0], [1, 0, 0, 0, 1], [0, 0, 1, 0, 0]])
g4_diff = nx.DiGraph(A)

fig, ax = plt.subplots(ncols=3, figsize=(20, 3))
for i, j in zip(ax, (g4, g4_same, g4_diff)):
    nx.draw_networkx(j, ax=i, pos={3: (0, 0), 0: (10, 0), 1: (20, 0), 2: (10, 5), 4: (5, 2.5)})

In [ ]:
def get_all_paths(g):
    # O(|E||V| + |V|^2 x log|V|)
    path_lens = nx.algorithms.shortest_paths.all_pairs_dijkstra_path_length(g)

    all_paths = set()
    for source, target_dict in path_lens:
        # print(source)
        for target, plen in target_dict.items():
            # print(target, plen)
            if plen > 0:
                all_paths.add((source, target))
    return all_paths
paths = get_all_paths(g4)
paths

In [ ]:
g4_same.nodes(data=False)

From [wikipedia](https://en.wikipedia.org/wiki/Shortest_path_problem):

<table align='left'>
    <tr>
        <th>type</th>
        <th>name</th>
        <th>weights</th>
        <th>complexity</th>
    </tr>
    <tr>
        <td>All-pair shortest path on directed weighted graphs</td>
        <td>Floyd-Warshall</td>
        <td>$R_{+}$</td>
        <td>$O(V^3)$</td>
    </tr>
    <tr>
        <td>Unweighted</td>
        <td>BFS</td>
        <td>$\{0, 1\}$</td>
        <td>$O(E+V)$</td>
    </tr>
    <tr>
        <td>Directed weighted</td>
        <td>Dijkstra's</td>
        <td>$R_{+}$</td>
        <td>$O(V^2)$ (with list), $O((E+V)log(V))$ (with binary heap, a.k.a Johnson's), $O(E + Vlog(log(V))$ (Thorup 2004)</td>
    </tr>
    <tr>
        <td>Directed weighted</td>
        <td>Bellman-Ford</td>
        <td>No negative cycles</td>
        <td> $O(VE)$</td>
    </tr>
</table>

In [ ]:
display(*get_paths_for_nodes(g4, [1, 2]))

In [ ]:
d4, _ = get_paths_for_nodes(g4, [0, 1, 2, 3])
d4_same, _ = get_paths_for_nodes(g4_same, [0, 1, 2, 3])
d4_diff, _ = get_paths_for_nodes(g4_diff, [0, 1, 2, 3])
display(
    d4, d4_same, d4_diff, np.all((d4 > 0) == (d4_same > 0)),
    np.all((d4 > 0) == (d4_diff > 0))
)

In [ ]:
d4_bfs = get_paths_for_nodes_bfs(g4, [0, 1, 2, 3])
np.all(d4_bfs == d4)

In [ ]:
subgraph = g4.edge_subgraph([(0, 1), (2, 3)])
nx.draw_networkx(subgraph, pos={3: (0, 0), 0: (10, 0), 1: (20, 0), 2: (10, 5), 4: (5, 2.5)})

In [ ]:
edges = [
    (6, 7), (7, 8), (6, 3), (7, 4), (8, 5),
    (3, 4), (4, 5), (3, 0), (4, 1), (5, 2),
    (0, 1), (1, 2),
]
rev_edges = [(j, i) for i, j in edges]
edges = edges + rev_edges
edges = edges + [
    (0, 'i0'), ('o0', 0), (1, 'i1'), ('o1', 1),
    (2, 'i2'), ('o2', 2), (3, 'i3'), ('o3', 3),
    (5, 'i5'), ('o5', 5), (6, 'i6'), ('o6', 6),
    (7, 'i7'), ('o7', 7), (8, 'i8'), ('o8', 8),
]

# cartesian frame, i.e. (x, y) pairs, with origin at bottom left
pos = {
    0: (0, 0), 1: (1, 0), 2: (2, 0), 3: (0, 1),
    4: (1, 1), 5: (2, 1), 6: (0, 2), 7: (1, 2),
    8: (2, 2), 'i0': (-.4, -.8), 'o0': (-.8, -.4), 
    'i1': (.8, -1), 'o1': (1.2, -1), 
    'i2': (2.6, -.4), 'o2': (2.4, -.6),
    'i3': (-1, .8), 'o3': (-1, 1.2),
    'i5': (3, .8), 'o5': (3, 1.2),
    'i6': (-.4, 2.6), 'o6': (-.6, 2.4),
    'i7': (.8, 3), 'o7': (1.2, 3),
    'i8': (2.4, 2.6), 'o8': (2.6, 2.4),
}
super_graph = nx.DiGraph(edges)
nx.draw_networkx(super_graph, pos=pos)

In [ ]:
route = Route(
    super_graph,
    NetList([('s1', ['t11', 't12']), ('s2', ['t21', 't22'])]),
    set([
        ('o0', 0), (0, 1), (1, 'i1'), (0, 3), (3, 'i3'), ('o8', 8), (8, 5),
        (5, 2), (2, 'i2'), (8, 7), (7, 'i7')
        
    ]),
    dict(s1='o0', t11='i1', t12='i3', s2='o8', t21='i2', t22='i7'),
    pos
)
_, axes = plt.subplots(nrows=1, ncols=3, figsize=(25, 12))
route.draw(axes=axes)
route.validate(), route.score()

# Test cases

In [ ]:
# (1, 4) is a dangling node
route = Route(
    super_graph,
    NetList([('s1', ['t1']),]),
    set([('o0', 0), (0, 1), (1, 2), (2, 'i2'), (1, 4)]),  # edge_list
    dict(s1='o0', t1='i2'),  # node_map
    pos
)
# _, axes = plt.subplots(nrows=1, ncols=3, figsize=(25, 12))
# route.draw(axes=axes)
assert route.validate() is False
route.score()

In [ ]:
# there is a loop in the wires
route = Route(
    super_graph,
    NetList([('s1', ['t1']),]),
    set([('o0', 0), (0, 3), (3, 4), (3, 6), (6, 7), (4, 7), (7, 'i7')]),  # edge_list
    dict(s1='o0', t1='i7'),  # node_map
    pos
)
# _, axes = plt.subplots(nrows=1, ncols=3, figsize=(25, 12))
# route.draw(axes=axes)
assert route.validate() is False
route.score()

In [ ]:
# there is a loop in the wires
route = Route(
    super_graph,
    NetList([('s1', ['t1']),]),
    set([('o0', 0), (0, 3), (3, 4), (3, 6), (6, 7), (7, 4), (7, 'i7')]),  # edge_list
    dict(s1='o0', t1='i7'),  # node_map
    pos
)
# _, axes = plt.subplots(nrows=1, ncols=3, figsize=(25, 12))
# route.draw(axes=axes)
assert route.validate() is False
route.score()

In [ ]:
super_graph.number_of_edges()